# Meneco Preview

This notebook is a preview demo of [meneco](https://pypi.python.org/pypi/meneco/) combined with the [IPython](http://ipython.org/) interactive environment. This demo assumes you have meneco 1.5.3 already installed.  

First you need to import the necessary modules ...

In [1]:
from pyasp.term import *
from urllib.request import urlopen
from __meneco__ import query, utils, sbml

Next, you can load a draft network from an sbml file and convert it into logic facts ...

In [2]:
draft_sbml= urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/ectocyc.sbml')
draftnet = sbml.readSBMLnetwork(draft_sbml, 'draft')    

load the seeds ...

In [3]:
seeds_sbml = urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/seeds.sbml')
seeds = sbml.readSBMLseeds(seeds_sbml)

and load the targets ...

In [4]:
targets_sbml = urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/targets.sbml')
targets = sbml.readSBMLtargets(targets_sbml)

Then you can check the draft network for unproducible targets ...

In [5]:
unproducible = TermSet()
model = query.get_unproducible(draftnet, targets, seeds)
for a in model:
    target = str(a)[13:]
    t      = String2TermSet(target)
    unproducible = unproducible.union(t)
unproducible = TermSet(unproducible)
utils.print_met(unproducible.to_list())

  "CPD__45__8121"
  "GLT"
  "CIT"
  "ILE"
  "D__45__ALANINE"
  "OLEATE__45__CPD"
  "HOMO__45__SER"
  "DOCOSANOATE"
  "PHE"
  "STEARIC_ACID"
  "HIS"
  "ARACHIDONIC_ACID"
  "LINOLENIC_ACID"
  "SER"
  "PRO"
  "_4__45__AMINO__45__BUTYRATE"
  "TYR"
  "GLN"
  "CPD__45__9247"
  "LEU"
  "ASN"
  "CPD__45__14292"
  "MET"
  "L__45__ORNITHINE"
  "CPD__45__8119"
  "SUC"
  "TRP"
  "VAL"
  "LYS"
  "CPD__45__7836"
  "ARACHIDIC_ACID"
  "_5Z8Z11Z14Z17Z__45__EICOSAPENTAENOATE"
  "CPD__45__13814"
  "GLYCERATE"
  "GLY"
  "GLUTATHIONE"
  "PALMITATE"
  "THREO__45__DS__45__ISO__45__CITRATE"
  "GLC"
  "CYS"
  "CPD__45__9245"
  "ARG"
  "CPD__45__8120"
  "THR"
  "L__45__ASPARTATE"
  "L__45__ALPHA__45__ALANINE"
  "LINOLEIC_ACID"
  "GLYCOLLATE"
  "GLYCEROL"
  "CPD__45__8117"
  "MANNITOL"


You can load another reaction network like metacyc repair data base...

In [6]:
repair_sbml = urlopen('https://raw.githubusercontent.com/bioasp/meneco/master/Ectodata/metacyc_16-5.sbml')
repairnet = sbml.readSBMLnetwork(repair_sbml, 'repairnet')

and combine the draft network with the repair database ...

In [7]:
combinet = draftnet
combinet = TermSet(combinet.union(repairnet))

and then check for which targets producibilty cannot be restored even with the combined networks ...

In [8]:
unrepairable = TermSet()
model = query.get_unproducible(combinet, targets, seeds)
for a in model:
    target = str(a)[13:]
    t      = String2TermSet(target)
    unrepairable = unrepairable.union(t)
unrepairable = TermSet(unrepairable)
utils.print_met(unrepairable)

  "CPD__45__8119"
  "CPD__45__14292"
  "CPD__45__13814"
  "DOCOSANOATE"


and for which targets the production paths are repairable ...

In [9]:
repairable = unproducible.difference(unrepairable)
utils.print_met(repairable)

  "CPD__45__8121"
  "GLT"
  "CIT"
  "ILE"
  "D__45__ALANINE"
  "OLEATE__45__CPD"
  "HOMO__45__SER"
  "PHE"
  "STEARIC_ACID"
  "HIS"
  "ARACHIDONIC_ACID"
  "LINOLENIC_ACID"
  "SER"
  "PRO"
  "_4__45__AMINO__45__BUTYRATE"
  "TYR"
  "GLN"
  "CPD__45__9247"
  "LEU"
  "ASN"
  "MET"
  "L__45__ORNITHINE"
  "SUC"
  "TRP"
  "VAL"
  "LYS"
  "CPD__45__7836"
  "ARACHIDIC_ACID"
  "_5Z8Z11Z14Z17Z__45__EICOSAPENTAENOATE"
  "GLYCERATE"
  "GLY"
  "GLUTATHIONE"
  "PALMITATE"
  "THREO__45__DS__45__ISO__45__CITRATE"
  "GLC"
  "CYS"
  "CPD__45__9245"
  "ARG"
  "CPD__45__8120"
  "THR"
  "L__45__ASPARTATE"
  "L__45__ALPHA__45__ALANINE"
  "LINOLEIC_ACID"
  "GLYCOLLATE"
  "GLYCEROL"
  "CPD__45__8117"
  "MANNITOL"


You can compute the essential reactions for the repairable target ...

In [10]:
essential_reactions = TermSet()
for t in repairable:
      single_target = TermSet()
      single_target.add(t)
      print('\nComputing essential reactions for', t,'...', end=' ')
      essentials = query.get_intersection_of_completions(draftnet, repairnet, seeds, single_target)
      print('done.')
      print(' ',len(essentials), 'essential reactions found:')
      utils.print_met(essentials.to_list())
      essential_reactions = essential_reactions.union(essentials)
essential_reactions= TermSet(essential_reactions)
print('\nOverall',len(essential_reactions), 'essential reactions found.')
utils.print_met(essential_reactions)


Computing essential reactions for target("CPD__45__8121") ... done.
  0 essential reactions found:

Computing essential reactions for target("GLT") ... done.
  0 essential reactions found:

Computing essential reactions for target("CIT") ... done.
  0 essential reactions found:

Computing essential reactions for target("ILE") ... done.
  0 essential reactions found:

Computing essential reactions for target("D__45__ALANINE") ... done.
  0 essential reactions found:

Computing essential reactions for target("OLEATE__45__CPD") ... done.
  5 essential reactions found:
  "RXN__45__9520"
  "_4__46__2__46__1__46__59__45__RXN"
  "_1__46__14__46__19__46__2__45__RXN"
  "RXN__45__9655"
  "_3__46__1__46__2__46__14__45__RXN"

Computing essential reactions for target("HOMO__45__SER") ... done.
  0 essential reactions found:

Computing essential reactions for target("PHE") ... done.
  0 essential reactions found:

Computing essential reactions for target("STEARIC_ACID") ... done.
  3 essential reac

You can compute a completion of minimal size suitable to produce all repairable targets ...

In [11]:
models =  query.get_minimal_completion_size(draftnet, repairnet, seeds, repairable)
optimum = models[0].score[0]
print('  minimal size =',optimum)
utils.print_met(models[0].to_list())

  minimal size = 45
  "RXN__45__13435"
  "RXN__45__9537"
  "RXN__45__12766"
  "RXN__45__9557"
  "RXN__45__9520"
  "PHOSPHOGLYCERATE__45__PHOSPHATASE__45__RXN"
  "GLUTDECARBOX__45__RXN"
  "RXN__45__12968"
  "_3__46__1__46__2__46__14__45__RXN"
  "_1__46__14__46__19__46__2__45__RXN"
  "HISTIDPHOS__45__RXN"
  "FORMYLMETHIONINE__45__DEFORMYLASE__45__RXN"
  "RXN__45__9550"
  "RXN__45__10727"
  "_3__46__4__46__19__46__7__45__RXN"
  "RXN__45__12777"
  "_3__46__4__46__14__46__1__45__RXN"
  "RXN__45__9549"
  "RXN__45__8347"
  "PREPHENATEDEHYDROG__45__RXN"
  "HISTCYCLOHYD__45__RXN"
  "RXN__45__12755"
  "RXN__45__8349"
  "RXN__45__9655"
  "MANNITOL__45__1__45__PHOSPHATASE__45__RXN"
  "_4__46__2__46__1__46__61__45__RXN"
  "RXN__45__9533"
  "_4__46__2__46__1__46__59__45__RXN"
  "_3__46__4__46__13__46__5__45__RXN"
  "SHIKIMATE__45__5__45__DEHYDROGENASE__45__RXN"
  "RXN__45__12971"
  "RXN__45__13242"
  "RXN__45__13261"
  "RXN__45__10663"
  "RXN__45__9558"
  "MALONYL__45__ACPDECARBOX__45__RXN"
  "_4__4

We can compute the common reactions in all completion with a given size ...

In [12]:
model = query.get_intersection_of_optimal_completions(draftnet, repairnet, seeds, repairable,  optimum)
utils.print_met(model.to_list())    

  "RXN__45__13435"
  "RXN__45__9537"
  "RXN__45__12766"
  "RXN__45__9557"
  "RXN__45__9520"
  "GLUTDECARBOX__45__RXN"
  "RXN__45__12968"
  "_3__46__1__46__2__46__14__45__RXN"
  "_1__46__14__46__19__46__2__45__RXN"
  "HISTIDPHOS__45__RXN"
  "RXN__45__9550"
  "RXN__45__10727"
  "RXN__45__12777"
  "RXN__45__9549"
  "HISTCYCLOHYD__45__RXN"
  "MANNITOL__45__1__45__PHOSPHATASE__45__RXN"
  "_4__46__2__46__1__46__61__45__RXN"
  "RXN__45__9533"
  "_4__46__2__46__1__46__59__45__RXN"
  "RXN__45__12971"
  "RXN__45__13242"
  "RXN__45__13261"
  "RXN__45__9558"
  "_4__46__2__46__1__46__58__45__RXN"
  "RXN__45__9634"
  "RXN__45__9548"
  "RXN__45__9555"
  "RXN__45__12969"
  "RXN__45__9673"
  "RXN__45__8346"
  "RXN__45__9655"


We can compute the union all completion with a given size ...

In [13]:
model = query.get_union_of_optimal_completions(draftnet, repairnet, seeds, repairable, optimum)
utils.print_met(model.to_list())

  "_4__45__COUMARATE__45____45__COA__45__LIGASE__45__RXN"
  "RXN__45__13435"
  "RXN__45__9537"
  "RXN__45__12487"
  "RXN__45__12361"
  "RXN__45__12766"
  "RXN__45__9557"
  "RXN__45__9520"
  "_3__46__4__46__13__46__17__45__RXN"
  "PHOSPHOGLYCERATE__45__PHOSPHATASE__45__RXN"
  "RXN__45__13431"
  "FATTY__45__ACID__45__SYNTHASE__45__RXN"
  "TYROSINE__45__AMINOTRANSFERASE__45__RXN"
  "ADENYLYLSULFATE__45__REDUCTASE__45__RXN"
  "GLUTDECARBOX__45__RXN"
  "RXN__45__12968"
  "_3__46__1__46__2__46__14__45__RXN"
  "_1__46__14__46__19__46__2__45__RXN"
  "HISTIDPHOS__45__RXN"
  "FORMYLMETHIONINE__45__DEFORMYLASE__45__RXN"
  "ADENOSYLHOMOCYSTEINASE__45__RXN"
  "RXN__45__9550"
  "RXN__45__10727"
  "_3__46__4__46__19__46__7__45__RXN"
  "RXN__45__12777"
  "ACP__45__S__45__ACETYLTRANSFER__45__RXN"
  "_3__46__4__46__14__46__1__45__RXN"
  "RXN__45__8347"
  "RXN__45__9549"
  "HISTCYCLOHYD__45__RXN"
  "PREPHENATEDEHYDROG__45__RXN"
  "RXN__45__12755"
  "RXN__45__12794"
  "_3__45__PHOSPHOGLYCERATE__45__PHOSPH

And finally compute all (for this notebook we print only the first three) completions with a given size ...

In [14]:
models = query.get_optimal_completions(draftnet, repairnet, seeds, repairable, optimum)
for i in range(0,3):
   print('Completion '+str(i+1)+':')
   utils.print_met(models[i].to_list())


Completion 1:
  "_4__45__COUMARATE__45____45__COA__45__LIGASE__45__RXN"
  "RXN__45__13435"
  "RXN__45__9537"
  "RXN__45__12361"
  "RXN__45__12766"
  "RXN__45__9557"
  "RXN__45__9520"
  "TYROSINE__45__AMINOTRANSFERASE__45__RXN"
  "GLUTDECARBOX__45__RXN"
  "RXN__45__12968"
  "_3__46__1__46__2__46__14__45__RXN"
  "_1__46__14__46__19__46__2__45__RXN"
  "HISTIDPHOS__45__RXN"
  "ADENOSYLHOMOCYSTEINASE__45__RXN"
  "RXN__45__9550"
  "RXN__45__10727"
  "RXN__45__12777"
  "RXN__45__9549"
  "_3__45__PHOSPHOGLYCERATE__45__PHOSPHATASE__45__RXN"
  "RXN__45__8347"
  "PREPHENATEDEHYDROG__45__RXN"
  "HISTCYCLOHYD__45__RXN"
  "RXN__45__12755"
  "RXN__45__7968"
  "RXN__45__8349"
  "RXN__45__701"
  "MANNITOL__45__1__45__PHOSPHATASE__45__RXN"
  "_4__46__2__46__1__46__61__45__RXN"
  "_4__46__3__46__1__46__23__45__RXN"
  "RXN__45__9533"
  "_4__46__2__46__1__46__59__45__RXN"
  "RXN__45__12971"
  "RXN__45__13242"
  "RXN__45__13261"
  "RXN__45__10663"
  "RXN__45__9558"
  "_4__46__2__46__1__46__58__45__RXN"
  "R

That's all folks!